In [3]:
import requests
from bs4 import BeautifulSoup
import re

In [4]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}

In [24]:
r = requests.get(f"https://www.amazon.com/s?i=computers&rh=n%3A565108&page=1", headers=headers)
r.content

b'<!--\n        To discuss automated access to Amazon data please contact api-services-support@amazon.com.\n        For information about migrating to our APIs refer to our Marketplace APIs at https://developer.amazonservices.com/ref=rm_5_sv, or our Product Advertising API at https://affiliate-program.amazon.com/gp/advertising/api/detail/main.html/ref=rm_5_ac for advertising use cases.\n-->\n<!doctype html>\n<html>\n<head>\n  <meta charset="utf-8">\n  <meta http-equiv="x-ua-compatible" content="ie=edge">\n  <meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no">\n  <title>Sorry! Something went wrong!</title>\n  <style>\n  html, body {\n    padding: 0;\n    margin: 0\n  }\n\n  img {\n    border: 0\n  }\n\n  #a {\n    background: #232f3e;\n    padding: 11px 11px 11px 192px\n  }\n\n  #b {\n    position: absolute;\n    left: 22px;\n    top: 12px\n  }\n\n  #c {\n    position: relative;\n    max-width: 800px;\n    padding: 0 40px 0 0\n  }\n\n  #e, #f {\n    hei

## Amazon Requests

In [42]:
def amazon_laptop_page(page_num):
    r = requests.get(f"https://www.amazon.com/s?i=computers&rh=n%3A565108&fs=true&page=2", headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    #print(soup.content)
    links = soup.html.body.select(".a-link-normal.s-underline-text.s-underline-link-text.s-link-style.a-text-normal")[3:]
    #print(links)
    names_a = links[::2]
    prices_a = links[1::2]
    hrefs = [n['href'] for n in names_a]
    names = [n.text for n in names_a]
    prices = [p.text for p in prices_a]
    return (hrefs, names, prices)

In [13]:
amazon_laptop_page(2)

None
[]


([], [], [])

In [50]:
def get_website_links(website_func, num_pages, file_name):
    with open(file_name, "w") as amazon_laptop_file:
        for page_num in range(num_pages):
            hrefs, names, prices = website_func(page_num)
            print(f"{len(hrefs)}, {len(names)}, {len(prices)}")
            for tup in zip(hrefs, names, prices):
                print(tup)
                amazon_laptop_file.write("\t".join(tup)+"\n")

In [ ]:
get_website_links(amazon_laptop_page, 10, "amazon_laptop_links.tsv")

## Newegg Requests

In [5]:
def newegg_laptop_page(page_num):
    price_pattern = r"[\d,]+\.\d{2}"
    page_link = f"https://www.newegg.com/Laptops-Notebooks/SubCategory/ID-32/Page-{page_num}"
    r = requests.get(page_link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    print(soup)
    soup = soup.html.body
    items = soup.select(".item-container")
    print(items)
    item_list = []
    for item in items:
        name_html = item.select(".item-title")[0]
        price_html = item.select(".price-current")
        price_match = re.findall(price_pattern, price_html[0].text)
        price = price_match[0] if price_match else None
        #print(f"{name_html},{price_html}")
        item_list.append((
            name_html['href'],
            name_html.text.replace("\t"," "),
            price if price else ""
        ))
    return item_list

def get_newegg_links(num_pages, file_name):
    with open(file_name, "w") as laptop_file:
        for page_num in range(1,num_pages+1):
            print(f"page {page_num}")
            for tup in newegg_laptop_page(page_num):
                print(tup)
                laptop_file.write("\t".join(tup)+"\n")

In [6]:
newegg_laptop_page(2)


<!DOCTYPE html>

<html lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>Are you a human?</title>
<meta content="IE=EDGE" http-equiv="X-UA-Compatible"/><meta charset="utf-8"/>
<link href="//c1.neweggimages.com/WebResource/Themes/2005/Nest/Newegg.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="https://fonts.googleapis.com/css?family=Open+Sans:400,300,300italic,400italic,700,700italic|Open+Sans+Condensed:300,300italic,700" rel="stylesheet" type="text/css"/>
<style type="text/css">
p{font-size:16px;color:#4d4d4d;padding:0;margin:0 0 5px}a img,a:hover img,a:visited img{border:0}.button-primary,.button-primary:focus,.button-primary:link,.button-primary:visited{font-family:'Open Sans Condensed','Arial Narrow','Helvetica Narrow',arial,helvetica,sans-serif;letter-spacing:1px;font-size:14px;font-weight:700;font-stretch:condensed;text-align:center;text-decoration:none;cursor:pointer;border-radius:4px;border:2px solid #E68626;display:inline-block;padding:9px 15px;margin:15

[]

In [49]:
get_newegg_links(10, "newegg_laptop_links.tsv")

page 1
page 2
page 3
page 4
page 5
page 6
page 7
page 8
page 9
page 10


# Getting Data From Product Pages

In [1]:
import pandas as pd

In [2]:
newegg_df = pd.read_csv("newegg_laptop_links.tsv", encoding = 'unicode_escape', sep="\t", header=None)

In [34]:
newegg_df

,0,1,2
0,https://www.newegg.com/pine-gray-asus-zenbook-...,"ASUS ZenBook 13 Ultra-Slim Laptop, 13.3"" OLED ...",849.99
1,https://www.newegg.com/black-lenovo-20uw0012us...,"Lenovo ThinkPad X12 Detachable Gen 1 Laptop, 1...",949.00
2,https://www.newegg.com/acer-sf514-55ta-77ww/p/...,Acer Intel EVO Laptop Swift 5 SF514-55TA-77WW ...,999.99
3,https://www.newegg.com/intel-nuc-m15-lapbc710/...,"Helix WorkPlex 1170EN Premium 15.6"" Touchscree...","1,249.99"
4,https://www.newegg.com/indie-black-asus-vivobo...,"ASUS VivoBook 15 OLED K513 Laptop, 15.6 OLED D...",919.99
...,...,...,...
3756,https://www.newegg.com/quiet-blue-asus-vivoboo...,ASUS Laptop VivoBook Pro K3500PC-DH59-CA Intel...,"1,098.22"
3757,https://www.newegg.com/p/1TS-000A-0CCK4,"Dell Inspiron 14 5000 Laptop Computer 14"" FHD ...",549.00
3758,https://www.newegg.com/p/1TS-000D-09G70,"HP Laptop 15.6"" Screen, 10th Gen Intel Core i7...",799.00
3759,https://www.newegg.com/p/2SN-0001-01B62,"Apple MacBook Air with Apple M1 Chip (13-inch,...","1,069.95"


In [5]:
newegg_df[:,0]

TypeError: '(slice(None, None, None), 0)' is an invalid key

## Get Data For Each Product

In [27]:
def get_product_data(link):
    r = requests.get(link, headers=headers)
    soup = BeautifulSoup(r.content, 'html.parser')
    ret = {}
    for row in soup.select("#product-details tr"):
        if len(row) == 2:
            rc = list(row.children)
            ret[rc[0].text.strip()] = rc[1].text
    return ret

In [28]:
get_product_data("https://www.newegg.com/pine-gray-asus-zenbook-um325ua-ds71-mainstream/p/N82E16834235800")

{'Best Seller Ranking': '#48 in Laptops / Notebooks',
 'Brand': 'ASUS',
 'Series': 'ZenBook',
 'Model': 'UM325UA-DS71',
 'Color': 'Pine Gray',
 'Operating System': 'Windows 10 Home 64-bit',
 'CPU': 'AMD Ryzen 7 5700U 1.80 GHz',
 'Screen': '13.3" OLED',
 'Memory': '8GB',
 'Storage': '512 GB PCIe SSD',
 'Graphics Card': 'AMD Radeon Graphics',
 'Video Memory': 'Shared system memory',
 'Dimensions (W x D x H)': '12.00" x 8.00" x 0.50"',
 'Weight': '2.50 lbs.',
 'CPU Type': 'AMD Ryzen 7 5000 Series',
 'CPU Speed': '5700U (1.80GHz)',
 'Turbo Frequency': 'Up to 4.30 GHz',
 'Number of Cores': '8-core Processor',
 'CPU L3 Cache': '8 MB',
 'Screen Size': '13.3"',
 'Touchscreen': 'No',
 'Wide Screen Support': 'Yes',
 'Display Type': 'Full HD',
 'Resolution': '1920 x 1080',
 'Panel': 'OLED',
 'LCD Features': '400 nits, Glossy',
 'GPU/VPU': 'AMD Radeon Graphics',
 'Graphic Type': 'Integrated Card',
 'SSD': '512 GB PCIe',
 'Memory Spec': '8 GB onboard',
 'Memory Slot (Total)': '0',
 'Memory Slot (Av

In [33]:
all_laptop_data = {}
for i, row in newegg_df.iterrows():
    all_laptop_data[row[1]] = get_product_data(row[0]) | {'Price': row[[2]]}
    print(f"{i}: {row[1][:15]}...")

0: ASUS ZenBook 13...
1: Lenovo ThinkPad...
2: Acer Intel EVO ...
3: Helix WorkPlex ...
4: ASUS VivoBook 1...
5: Helix WorkPlex ...
6: Helix WorkPlex ...
7: Lenovo Laptop T...
8: ASUS ZenBook Du...
9: ASUS ZenBook 13...
10: ASUS VivoBook 1...
11: MSI Laptop Crea...
12: ASUS Laptop L41...
13: MSI Laptop Crea...
14: Toshiba Dynaboo...
15: Lenovo Laptop T...
16: ASUS Laptop Zen...
17: MSI Laptop Crea...
18: ASUS VivoBook P...
19: HP Laptop 17-cp...
20: Lenovo Laptop T...
21: ASUS VivoBook P...
22: HP Laptop ProBo...
23: ASUS VivoBook S...
24: ASUS VivoBook 1...
25: HP Laptop ProBo...
26: ASUS ZenBook 14...
27: HP ProBook 430 ...
28: MSI Laptop Crea...
29: HP Laptop ProBo...
30: HP Laptop Pavil...
31: MSI Laptop Crea...
32: HP Laptop 250 G...
33: Lenovo ThinkPad...
34: HP Laptop 255 G...
35: HP ProBook 450 ...
36: HP Laptop ProBo...
37: ASUS ZenBook 13...
38: Lenovo ThinkPad...
39: Acer Intel EVO ...
40: Helix WorkPlex ...
41: ASUS VivoBook 1...
42: Helix WorkPlex ...
43: Helix WorkPlex ..

MissingSchema: Invalid URL 'i3505-A542BLK-PUS Notebook PC Computer': No schema supplied. Perhaps you meant http://i3505-A542BLK-PUS Notebook PC Computer?

In [55]:
all_laptop_data

{'ASUS ZenBook 13 Ultra-Slim Laptop, 13.3" OLED FHD NanoEdge Bezel Display, AMD Ryzen 7 5700U, 8GB LPDDR4X RAM, 512GB PCIe SSD, NumberPad, Wi-Fi 5, Windows 10 Home, Pine Grey, UM325UA-DS71': {'Best Seller Ranking': '#48 in Laptops / Notebooks',
  'Brand': 'ASUS',
  'Series': 'ZenBook',
  'Model': 'UM325UA-DS71',
  'Color': 'Pine Gray',
  'Operating System': 'Windows 10 Home 64-bit',
  'CPU': 'AMD Ryzen 7 5700U 1.80 GHz',
  'Screen': '13.3" OLED',
  'Memory': '8GB',
  'Storage': '512 GB PCIe SSD',
  'Graphics Card': 'AMD Radeon Graphics',
  'Video Memory': 'Shared system memory',
  'Dimensions (W x D x H)': '12.00" x 8.00" x 0.50"',
  'Weight': '2.50 lbs.',
  'CPU Type': 'AMD Ryzen 7 5000 Series',
  'CPU Speed': '5700U (1.80GHz)',
  'Turbo Frequency': 'Up to 4.30 GHz',
  'Number of Cores': '8-core Processor',
  'CPU L3 Cache': '8 MB',
  'Screen Size': '13.3"',
  'Touchscreen': 'No',
  'Wide Screen Support': 'Yes',
  'Display Type': 'Full HD',
  'Resolution': '1920 x 1080',
  'Panel': 'O

In [62]:
newegg_all_laptop_df = pd.DataFrame(all_laptop_data).transpose()

In [64]:
newegg_all_laptop_df.to_csv("all_laptop_df.csv")

In [57]:
import json
with open("all_notebook_data.json", "w") as nb_data:
    nb_data.write(json.dumps(all_laptop_data))

TypeError: Object of type Series is not JSON serializable